In [33]:
import pandas as pd
import numpy as np

input_data= pd.read_csv('train.csv')
input_data.dropna(axis=0,subset=['SalePrice'],inplace = True)

In [34]:
y_data = input_data.SalePrice
x_data = input_data.drop(['Id','SalePrice'],axis=1)

In [35]:
x_numeric = x_data.select_dtypes(exclude=['object'])
x_text = x_data.select_dtypes(include = ['object'])

In [54]:
from sklearn.preprocessing import Imputer
def impute_df(DF):
    imputer = Imputer()
    columns = DF.columns
    index = DF.index
    DF_imputed = pd.DataFrame(imputer.fit_transform(DF))
    DF_imputed.columns = columns
    DF_imputed.index = index
    return DF_imputed

x_imputed = impute_df(x_numeric)
x_imputed.isnull().any()

MSSubClass       False
LotFrontage      False
LotArea          False
OverallQual      False
OverallCond      False
YearBuilt        False
YearRemodAdd     False
MasVnrArea       False
BsmtFinSF1       False
BsmtFinSF2       False
BsmtUnfSF        False
TotalBsmtSF      False
1stFlrSF         False
2ndFlrSF         False
LowQualFinSF     False
GrLivArea        False
BsmtFullBath     False
BsmtHalfBath     False
FullBath         False
HalfBath         False
BedroomAbvGr     False
KitchenAbvGr     False
TotRmsAbvGrd     False
Fireplaces       False
GarageYrBlt      False
GarageCars       False
GarageArea       False
WoodDeckSF       False
OpenPorchSF      False
EnclosedPorch    False
3SsnPorch        False
ScreenPorch      False
PoolArea         False
MiscVal          False
MoSold           False
YrSold           False
dtype: bool

In [37]:
cardinalities = [x_text[col].nunique() for col in x_text.columns]
print(cardinalities)


[5, 2, 2, 4, 4, 2, 5, 3, 25, 9, 8, 5, 8, 6, 8, 15, 16, 4, 4, 5, 6, 4, 4, 4, 6, 6, 6, 5, 2, 5, 4, 7, 5, 6, 3, 5, 5, 3, 3, 4, 4, 9, 6]


In [38]:
max_cardinality = 10
high_card_colsls = [col for col in x_text.columns if x_text[col].nunique()> max_cardinality]

In [39]:
x_text = x_text.drop(high_card_cols,axis=1)
print(high_card_cols)

['Neighborhood', 'Exterior1st', 'Exterior2nd']


In [76]:
x_text = pd.get_dummies(x_text)
x_train = pd.concat([x_imputed,x_text],axis=1)
x_train.shape

(1460, 232)

In [73]:
test_data = pd.read_csv('test.csv')
test_data.drop('Id',axis=1)
test_numeric = test_data.select_dtypes(exclude=['object'])
test_text = test_data.select_dtypes(include=['object'])


In [74]:
test_imputed = impute_df(test_numeric)
high_card_cols = [col for col in test_text.columns if test_text[col].nunique() > max_cardinality]
test_text = test_text.drop(high_card_cols,axis=1)
test_text = pd.get_dummies(test_text)
test_data = pd.concat([test_imputed,test_text],axis=1)

In [75]:
test_data.shape

(1459, 218)

In [77]:
test_data.isnull().any()

Id                       False
MSSubClass               False
LotFrontage              False
LotArea                  False
OverallQual              False
OverallCond              False
YearBuilt                False
YearRemodAdd             False
MasVnrArea               False
BsmtFinSF1               False
BsmtFinSF2               False
BsmtUnfSF                False
TotalBsmtSF              False
1stFlrSF                 False
2ndFlrSF                 False
LowQualFinSF             False
GrLivArea                False
BsmtFullBath             False
BsmtHalfBath             False
FullBath                 False
HalfBath                 False
BedroomAbvGr             False
KitchenAbvGr             False
TotRmsAbvGrd             False
Fireplaces               False
GarageYrBlt              False
GarageCars               False
GarageArea               False
WoodDeckSF               False
OpenPorchSF              False
                         ...  
GarageCond_Gd            False
GarageCo

In [81]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
final_train,final_test = x_train.align(test_data,join = 'inner',axis =1)
final_train.shape

(1460, 217)

In [82]:
final_test.shape

(1459, 217)

In [84]:
y_data.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [86]:
linreg.fit(final_train,y_data)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [97]:
y_pred = linreg.predict(final_test)
for i in range(0,len(y_pred)):
    if y_pred[i]<=0:
        y_pred[i] = y_pred[i]*(-1)
        print(y_pred[i])
my_sol = pd.DataFrame({'Id':range(1461,2920),'SalePrice':y_pred})
my_sol.to_csv('submission.csv',index=False)

6499.88403477
21459.2824165
8156.12685435
